In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard


---
# Project 2 - Part 1
For the first part, use [the pretrained Bert model from hugging face](https://huggingface.co/transformers/task_summary.html#question-answering) and feed it with the five 300 words long sections from the book of your choice.
These sections should be selected so they are: introducing the protagonist(s), the antagonist, the crime and crime scene, any significant evidence, and the resolution of the crime/a narrative that presents the case against the perpetrator.
The questions you should ask are about the identity and characteristics of the protagonist, antagonist/perpetrator, the nature and the setting of the crime or crime scene, the evidence, and the case against the perpetrator.
Document the questions, ask the questions and document the specificity and accuracy of the results.
Then, feed the whole novel as a context, and repeat the questions. Document the results

In [2]:

import os
from pprint import pprint
from transformers import pipeline
from transformers import DistilBertTokenizerFast, TFDistilBertForQuestionAnswering
from custom_squad import *  # creating and loading in custom squad questions and answers
from question_answering import *  # automating the question answering process


2022-10-31 10:27:08.003220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 10:27:08.064703: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-31 10:27:08.081147: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-31 10:27:08.333705: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

---
# Load in Normal Distilbert

In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
generator = pipeline(task="question-answering", model=model, tokenizer=tokenizer)

2022-10-31 10:27:09.564557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 10:27:09.576796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 10:27:09.576876: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 10:27:09.577216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

---
# Load in Custom Questions

In [4]:
samples = create_samples()

In [5]:
# peek at a samples
print(f'Num Samples: {len(samples)}')
print(f'Sample Keys: {samples[0].keys()}')


Num Samples: 25
Sample Keys: dict_keys(['id', 'title', 'context', 'question', 'answers'])


In [6]:
print(f'sample[0]:')
pprint(samples[0])


sample[0]:
{'answers': {'answer_start': [608], 'text': ['practitioner']},
 'context': 'Mr. Sherlock Holmes, who was usually very late in the mornings, '
            'save upon those not infrequent occasions when he was up all '
            'night, was seated at the breakfast table. I stood upon the '
            'hearth-rug and picked up the stick which our visitor had left '
            'behind him the night before. It was a fine, thick piece of wood, '
            'bulbous-headed, of the sort which is known as a \\"Penang '
            'lawyer.\\" Just under the head was a broad silver band nearly an '
            'inch across. \\"To James Mortimer, M.R.C.S., from his friends of '
            'the C.C.H.,\\" was engraved upon it, with the date \\"1884.\\" It '
            'was just such a stick as the old-fashioned family practitioner '
            'used to carry—dignified, solid, and reassuring. \\"Well, Watson, '
            'what do you make of it?\\" Holmes was sitting with his b

---
# Test QA

In [7]:
# Test pipeline generator
_ = qa(generator, samples[0], verbose=True)


QUESTION: "What is the guests occupation?"
GROUND TRUTH: "practitioner"
PREDICTED: "Watson, what do you make"
SCORE: "4.443894431460649e-05"


---
# Non-Finetuned Question vs Answers With 500 Word Contexts

In [8]:
# Run all of the samples through the generator
predicted = list(map(lambda s: qa(generator, s, verbose=False), samples))

# Package the sample with the predicted
questions_answers = list(zip(samples, predicted))


In [9]:
n = len(questions_answers)

# print predictions to console for quick inspection
for i, answer in enumerate(questions_answers):
    print_answer(i, questions_answers[i], n=n)

# write predictions to a file for later
with open(os.path.join('results', 'non-fine-tuned-500-word-context.txt'), 'w') as f:
    for i, answer in enumerate(questions_answers):
        print_answer(i, questions_answers[i], n=n, file=f)

****************************************************************************************************

QUESTION #0:
"What is the guests occupation?"

GROUND TRUTH ANSWER:
"practitioner"

PREDICTED ANSWER (score=4.443894431460649e-05):
"Watson, what do you make"

****************************************************************************************************

QUESTION #1:
"How did Sherlock Holmes see Watson?"

GROUND TRUTH ANSWER:
"well-polished, silver-plated coffee-pot"

PREDICTED ANSWER (score=4.665433516493067e-05):
"Watson, what do you make"

****************************************************************************************************

QUESTION #2:
"What did Holmes and Watson inspect?"

GROUND TRUTH ANSWER:
"visitor's stick"

PREDICTED ANSWER (score=4.6880879381205887e-05):
"Watson, what do you make"

****************************************************************************************************

QUESTION #3:
"Where was Sherlock Holmes sitting?"

GROUND TRUTH ANSWER:

---
# Non-Finetuned Question vs Answers With Whole Book As Context

In [10]:
# read in whole book
context = clean_context(os.path.join('text', 'the-hound-of-the-baskervilles.txt'))

# change samples contexts to full book
samples_large_context = samples.copy()
for i in range(len(samples_large_context)):
    samples_large_context[i]['context'] = context

In [37]:
# Run all of the samples through the generator
predicted = list(map(lambda s: qa(generator, s, verbose=False), samples_large_context))

# Package the sample with the predicted
questions_answers = list(zip(samples_large_context, predicted))


In [39]:
n = len(questions_answers)

# print predictions to console for quick inspection
for i, answer in enumerate(questions_answers):
    print_answer(i, questions_answers[i], n=n)

# write predictions to a file for later
with open(os.path.join('results', 'non-fine-tuned-book-context.txt'), 'w') as f:
    for i, answer in enumerate(questions_answers):
        print_answer(i, questions_answers[i], n=n, file=f)

****************************************************************************************************

QUESTION #0:
"What is the guests occupation?"

GROUND TRUTH ANSWER:
"practitioner"

PREDICTED ANSWER (score=3.951425605919212e-05):
"Stapleton has"

****************************************************************************************************

QUESTION #1:
"How did Sherlock Holmes see Watson?"

GROUND TRUTH ANSWER:
"well-polished, silver-plated coffee-pot"

PREDICTED ANSWER (score=2.329632297914941e-05):
"our minds the grim and grey impression which had been left upon both of"

****************************************************************************************************

QUESTION #2:
"What did Holmes and Watson inspect?"

GROUND TRUTH ANSWER:
"visitor's stick"

PREDICTED ANSWER (score=2.3236951165017672e-05):
"our minds the grim and grey impression which had been left upon both of"

******************************************************************************************

In [11]:
generator(question='Who is the protagonist?', context=context)


{'score': 5.4886535508558154e-05,
 'start': 318350,
 'end': 318354,
 'answer': 'from'}

In [12]:
generator(question='Who is the perpetrator?', context=context)


{'score': 2.169448089262005e-05,
 'start': 318276,
 'end': 318354,
 'answer': 'and retaining a claim upon some proportion of his income. We cannot doubt from'}

In [13]:
generator(question='What is the setting of the crime?', context=context)


{'score': 0.00011635889677563682,
 'start': 318350,
 'end': 318354,
 'answer': 'from'}

In [14]:
generator(question='What is the evidence against the perpetrator?', context=context)


{'score': 2.8686723453574814e-05,
 'start': 318350,
 'end': 318354,
 'answer': 'from'}